In [ ]:
import pandas as pd, numpy as np
from matplotlib import pyplot as plt

import scipy.stats  as stats

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
import os
os.listdir('../input/mym5files/')

In [ ]:
# best = pd.read_csv("../input/accuracy-best-public-lbs/kkiller_first_public_notebook_under050_v5.csv")
# best = pd.read_csv("../input/mym5files/WA3lgb1keras_m103.csv")
# best = pd.read_csv("../input/mym5files/GM3lgb1keras_m104.csv")
# best = pd.read_csv("../input/mym5files/WA3lgb1keras_m97_no.csv")
# best = pd.read_csv("../input/mym5files/GM3lgb1keras_m99_no.csv")###

# best = pd.read_csv("../input/mym5files/GM3lgb1keras_m99_no.csv")
# best2 = pd.read_csv("../input/mym5files/GM3lgb1keras_m104.csv")

# best.iloc[:30490,1:]=(best.iloc[:30490,1:]+best2.iloc[:30490,1:])/2
# best = pd.read_csv("../input/mym5files/lgb3anddnn_fake_valid_day_to_day2/lgb3anddnn_fake_valid_day_to_day2.csv")###
# best = pd.read_csv("../input/mym5files/3of3perstoreAND1All800ANDvalid_day_to_day2.csv")


# best = pd.read_csv("../input/mym5files/lgbmsubm.csv")###
# best = pd.read_csv("../input/mym5files/lgbmsubm_GMnasScos.csv")###
best = pd.read_csv("../input/mym5files/GMmasmodels_v2.csv")###
# best = pd.read_csv("../input/mym5files/newEns6keras.csv")###
# best = pd.read_csv("../input/mym5files/newEnsCliped.csv")###
# best = pd.read_csv("../input/mym5files/lgbm_final_VER4again.csv")###
# best = pd.read_csv("../input/mym5files/newEns.csv")###
# best = pd.read_csv("../input/mym5files/GM3models_nofastai.csv")###
# best = pd.read_csv("../input/mym5files/GM3models_nolgbcos.csv")###
# best = pd.read_csv("../input/mym5files/GMmasmodels_v1.csv")###
# best = pd.read_csv("../input/mym5files/GMallmodels_v1.csv")


best.head()

In [ ]:
# best.iloc[30490:,1:]=best.iloc[:30490,1:].values
best.iloc[:30490,1:]=best.iloc[30490:,1:].values

In [ ]:
best.tail()

In [ ]:
# best.iloc[:,1:]=best.iloc[:,1:]*0.97#9#
best.head()

In [ ]:
c=0.04
best.iloc[:,1:]=best.iloc[:,1:].ewm(com=c,axis=1).mean().values

In [ ]:
best.head()

In [ ]:
sales = pd.read_csv("../input/m5-forecasting-uncertainty/sales_train_validation.csv")
salesids=sales.id.values
sales = pd.read_csv("../input/m5-forecasting-uncertainty/sales_train_evaluation.csv")
sales.id=salesids
sales.head()

In [ ]:
sub = best.merge(sales[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]], on = "id")
sub["_all_"] = "Total"
sub.shape

In [ ]:
sub.head()

## Different ratios for different aggregation levels

The higher the aggregation level, the more confident we are in the point prediction --> lower coef, relatively smaller range of quantiles

In [ ]:
qs = np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995])
qs.shape


def get_ratios(coef=0.15):
    qs2 = np.log(qs/(1-qs))*coef
    ratios = stats.norm.cdf(qs2)
    ratios /= ratios[4]
    ratios = pd.Series(ratios, index=qs)
    return ratios.round(3)

def get_ratios(coef=0.15):
    qs2 = np.log(qs/(1-qs))*coef
    ratios = stats.norm.cdf(qs2)
    ratios /= ratios[4]
    ratios = pd.Series(ratios, index=qs)
    return ratios.round(3)

def get_ratios(coef=0.15):
    qs2 = np.log(qs/(1-qs))*coef
    ratios = stats.norm.cdf(qs2)
    ratios /= ratios[4]
#     ratios[-2] *= 1.01
#     ratios[0] *= 0.99
    ratios[-1] *= 1.03
    ratios = pd.Series(ratios, index=qs)
    return ratios.round(3)

def get_ratios2(coef=0.15, a=1.2):
    qs2 = np.log(qs/(1-qs))*coef
    ratios = stats.skewnorm.cdf(qs2, a)
    ratios /= ratios[4]
    ratios[-1] *= 1.02
    ratios = pd.Series(ratios, index=qs)
    return ratios.round(3)

def get_ratios3(coef=0.15, c=0.5, s=0.1):
#     qs2 = np.log(qs/(1-qs))*coef
    qs2 = qs*coef
    ratios = stats.powerlognorm.ppf(qs2, c, s)
    ratios /= ratios[4]
    ratios[0] *= 0.25
    ratios[-1] *= 1.02
    ratios = pd.Series(ratios, index=qs)
    return ratios.round(3)

def widen(array, pc):
    #array : array of df
    #pc: per cent (0:100)
    array[array<1]=array[array<1] * (1 - pc/100)
    array[array>1]=array[array>1] * (1 + pc/100)
    return array
#coef between 0.05 and 0.24 is used, probably suboptimal values for now
# get_ratios2(coef=0.3)
# get_ratios3(coef=.7, c=0.1, s=0.65)
widen_coef=0.5#5
level_coef_dict = {"id": widen((get_ratios2(coef=0.3)+get_ratios3(coef=.3, c=0.04, s=0.9))/2, pc=widen_coef), "item_id": widen(get_ratios2(coef=0.18, a=0.4),pc=widen_coef),
                   "dept_id": widen(get_ratios(coef=0.04),widen_coef), "cat_id": widen(get_ratios(coef=0.03),widen_coef),
                   "store_id": widen(get_ratios(coef=0.035),widen_coef), "state_id": widen(get_ratios(coef=0.03),widen_coef), 
                   "_all_": widen(get_ratios(coef=0.025),widen_coef),
                   ("state_id", "item_id"): widen(get_ratios2(coef=0.21, a=0.75),pc=widen_coef),  ("state_id", "dept_id"): widen(get_ratios(coef=0.05),widen_coef),
                    ("store_id","dept_id") : widen(get_ratios(coef=0.07),widen_coef), ("state_id", "cat_id"): widen(get_ratios(coef=0.04),widen_coef),
                    ("store_id","cat_id"): widen(get_ratios(coef=0.055),widen_coef)
                  }

# #################################################
# def get_ratios(coef=0.15):
#     qs2 = np.log(qs/(1-qs))*coef
#     ratios = stats.norm.cdf(qs2)
#     ratios /= ratios[4]
# #     ratios[-2] *= 1.01
# #     ratios[0] *= 0.99
# #     ratios[-1] *= 1.03
#     ratios = pd.Series(ratios, index=qs)
#     return ratios.round(3)

# def get_ratios2(coef=0.15, a=1.2):
#     qs2 = np.log(qs/(1-qs))*coef
#     ratios = stats.skewnorm.cdf(qs2, a)
#     ratios /= ratios[4]
# #     ratios[-1] *= 1.02
#     ratios = pd.Series(ratios, index=qs)
#     return ratios.round(3)

# def get_ratios3(coef=0.15, c=0.5, s=0.1):
# #     qs2 = np.log(qs/(1-qs))*coef
#     qs2 = qs*coef
#     ratios = stats.powerlognorm.ppf(qs2, c, s)
#     ratios /= ratios[4]
#     ratios[0] *= 0.25
# #     ratios[-1] *= 1.02
#     ratios = pd.Series(ratios, index=qs)
#     return ratios.round(3)
# #################################################



# # level_coef_dict = {"id": (get_ratios2(coef=0.3)+get_ratios3(coef=.3, c=0.04, s=0.9))/2, "item_id": get_ratios2(coef=0.18, a=0.4),
# #                    "dept_id": get_ratios(coef=0.04), "cat_id": get_ratios(coef=0.03),
# #                    "store_id": get_ratios(coef=0.035), "state_id": get_ratios(coef=0.03), 
# #                    "_all_": get_ratios(coef=0.025),
# #                    ("state_id", "item_id"): get_ratios2(coef=0.21, a=0.75),  ("state_id", "dept_id"): get_ratios(coef=0.05),
# #                     ("store_id","dept_id") : get_ratios(coef=0.07), ("state_id", "cat_id"): get_ratios(coef=0.04),
# #                     ("store_id","cat_id"): get_ratios(coef=0.055)
# #                   }

# level_coef_dict = {"id": get_ratios(coef=0.3), "item_id": get_ratios(coef=0.19),
#                    "dept_id": get_ratios(coef=0.05), "cat_id": get_ratios(coef=0.035),
#                    "store_id": get_ratios(coef=0.04), "state_id": get_ratios(coef=0.035), "_all_": get_ratios(coef=0.025),
#                    ("state_id", "item_id"): get_ratios(coef=0.19),  ("state_id", "dept_id"): get_ratios(coef=0.05),
#                     ("store_id","dept_id") : get_ratios(coef=0.06), ("state_id", "cat_id"): get_ratios(coef=0.04),
#                     ("store_id","cat_id"): get_ratios(coef=0.05)
#                   }

In [ ]:
# (get_ratios2(coef=0.3)+get_ratios3(coef=.3, c=0.04, s=0.9))/2

In [ ]:
a=((get_ratios2(coef=0.3)+get_ratios3(coef=.3, c=0.04, s=0.9))/2).values
np.where(a==1)[0][0]

In [ ]:
widen(((get_ratios2(coef=0.3)+get_ratios3(coef=.3, c=0.04, s=0.9))/2), 1)

In [ ]:
a[a<1]*0.99,a[a>1]*1.01

In [ ]:
a

In [ ]:
# get_ratios2(coef=0.15, a=1.2)

In [ ]:
# get_ratios(coef=0.025)

In [ ]:
# def get_ratios3(coef=0.15, c=0.5, s=0.1):
# #     qs2 = np.log(qs/(1-qs))*coef
#     qs2 = qs*coef
#     ratios = stats.powerlognorm.ppf(qs2, c, s)
#     ratios /= ratios[4]
# #     ratios[0] *= 0.25
#     ratios[-1] *= 1.04
#     ratios = pd.Series(ratios, index=qs)
#     return ratios.round(3)
# get_ratios3(coef=.5, c=0.4, s=0.07)

In [ ]:
# get_ratios3(coef=.7, c=0.1, s=0.65)

In [ ]:
# plt.plot(abs(1-get_ratios(coef=0.025)))
# plt.show()
# get_ratios(coef=0.025)

In [ ]:
# 1.105*1.025

In [ ]:
# plt.plot(abs(1-get_ratios(coef=0.025)))
# plt.show()
# get_ratios(coef=0.025)

In [ ]:
# plt.plot(abs(1-get_ratios2(coef=0.21, a=0.75)))
# plt.show()
# get_ratios2(coef=0.21, a=0.75)

In [ ]:
# (30490+3049+7+3+10+3+1)*9

In [ ]:
# 30k, 1, 0.3
# 3000, 10, 0.19----0.18
# 7, 10, 0.05-----0.055
# 3, 10, 0.035
# 10, 10, 0.04
# 3, 3-4, 0.035
# 1, 1, 0.025
# 9000, 3-4, 0.19-----0.21
# 21, 3-4, 0.05---0.07
# 70, 1, 0.06---0.09
# 9, 3-4, 0.04
# 30, 1, 0.05

In [ ]:
# (30490+3049+7+3+10+1)*9

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# a=np.array([30000, 3000, 7, 3, 10, 3, 1, 9000, 21, 70, 9, 30])
# b=np.array([0.3, 0.17, 0.055, 0.035, 0.04, 0.035, 0.025, 0.22, 0.07, 0.09, 0.04, 0.05])
# plt.plot(a,b,'.')
# plt.show()
# plt.plot(a[a<3000],b[a<3000],'.')
# plt.show()

In [ ]:
# # def get_ratios2(coef=0.15, a=0.05):
# #     qs2 = np.log(qs/(1-qs))*coef
# #     ratios = stats.skewnorm.cdf(qs2, a)
# #     ratios /= ratios[4]
# #     ratios = pd.Series(ratios, index=qs)
# #     return ratios.round(3)

# # get_ratios2(coef=0.3)
# level_coef_dict["item_id"]

In [ ]:
# level_coef_dict["id"]

In [ ]:
# level_coef_dict["_all_"]

In [ ]:


def quantile_coefs(q, level):
    ratios = level_coef_dict[level]
               
    return ratios.loc[q].values

def get_group_preds(pred, level):
    df = pred.groupby(level)[cols].sum()
    q = np.repeat(qs, len(df))
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
    df[cols] *= quantile_coefs(q, level)[:, None]
    if level != "id":
        df["id"] = [f"{lev}_X_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    else:
        df["id"] = [f"{lev.replace('_validation', '')}_{q:.3f}_validation" for lev, q in zip(df[level].values, q)]
    df = df[["id"]+list(cols)]
    return df

def get_couple_group_preds(pred, level1, level2):
    df = pred.groupby([level1, level2])[cols].sum()
    q = np.repeat(qs, len(df))
    df = pd.concat([df]*9, axis=0, sort=False)
    df.reset_index(inplace = True)
    df[cols] *= quantile_coefs(q, (level1, level2))[:, None]
    df["id"] = [f"{lev1}_{lev2}_{q:.3f}_validation" for lev1,lev2, q in 
                zip(df[level1].values,df[level2].values, q)]
    df = df[["id"]+list(cols)]
    return df

levels = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "_all_"]
couples = [("state_id", "item_id"),  ("state_id", "dept_id"),("store_id","dept_id"),
                            ("state_id", "cat_id"),("store_id","cat_id")]
cols = [f"F{i}" for i in range(1, 29)]

df = []
for level in levels :
    df.append(get_group_preds(sub, level))
for level1,level2 in couples:
    df.append(get_couple_group_preds(sub, level1, level2))
df = pd.concat(df, axis=0, sort=False)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df,df] , axis=0, sort=False)
df.reset_index(drop=True, inplace=True)
df.loc[df.index >= len(df.index)//2, "id"] = df.loc[df.index >= len(df.index)//2, "id"].str.replace(
                                    "_validation$", "_evaluation")

df.shape

df.head()

# df.to_csv("submission01.csv", index = False)

In [ ]:
# salesGitem=sales.groupby('item_id')[['item_id']+[f"d_{i}" for i in range(914, 1914)]].sum()
salesGitem=sales.groupby('item_id')[['item_id']+[f"d_{i}" for i in range(942, 1942)]].sum()

In [ ]:
salesGitem.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
sales.sort_values('id',inplace=True)
sales.reset_index(drop=True, inplace = True)

In [ ]:
sales.iloc[:,-28:]

In [ ]:
# quantsales=sales.iloc[:,-1000:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T
# quantsales=sales.iloc[:,-366:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T
# quantsales=(sales.iloc[:,-28:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T).values
# quantsales=((sales.iloc[:,-366:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T +
# #             sales.iloc[:,-366-365:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T +    
#             sales.quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T
#             )/2)
# quantsales=np.mean(np.stack((sales.iloc[:,-366:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T ,
#             sales.iloc[:,-28:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T ,
# #             sales.iloc[:,-366-365:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T ,
# #             sales.quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T
#             )),0)
quantsales=np.average(np.stack((
            sales.iloc[:,-364:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T ,
#             sales.iloc[:,-2*366:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T ,
            sales.iloc[:,-28:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T ,
#             sales.iloc[:,-366-365:].quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T ,
#             sales.quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T
            )),axis=0,weights=[1,1.75])
quantsales.shape, quantsales[:6,:]#, quantsales.head()

In [ ]:
quantsales.shape, np.repeat(np.expand_dims(quantsales.reshape(quantsales.shape[0]*quantsales.shape[1], order='F'),-1),28,1).shape

In [ ]:
sales.iloc[:,-366:]

In [ ]:
sales.shape

In [ ]:
quantsales.shape

In [ ]:
quantsalesW=[]
for i in range(7):
#     quantsalesW.append(np.expand_dims((sales.iloc[:,np.arange(-28*13+i,0,7)].quantile(np.array(
#         [0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T).values.reshape(quantsales.shape[0]*quantsales.shape[1], order='F'),-1) )
    quantsalesW.append( 
        np.expand_dims(
        np.average(
            np.stack(
        ((sales.iloc[:,np.arange(-28*13+i,0,7)].quantile(np.array(
        [0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T).values.reshape(quantsales.shape[0]*quantsales.shape[1], order='F'),
        (sales.iloc[:,np.arange(-28*3+i,0,7)].quantile(np.array(
        [0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T).values.reshape(quantsales.shape[0]*quantsales.shape[1], order='F')) 
         ,-1   ),axis=-1), -1)
                      )
quantsalesW=np.hstack(quantsalesW)

In [ ]:
quantsalesW.shape

In [ ]:
quantsalesW

In [ ]:
# a=(sales.iloc[:,np.arange(-28*13+i,0,7)].quantile(np.array(
#         [0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T).values.reshape(quantsales.shape[0]*quantsales.shape[1], order='F')
# np.average(np.stack((a,a),-1),-1)#.shape

In [ ]:
quantsalesW

In [ ]:
quantsalesW=np.tile(quantsalesW,4)
quantsalesW.shape

In [ ]:
quantsalesW

In [ ]:
quantsalesW

In [ ]:
quantsales

In [ ]:
quantsales.shape

In [ ]:
quantsales

In [ ]:
medians=np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[:274410,0]] )== 0.5)[0]
notmedian=np.array([x for x in np.arange(274410) if x not in medians])

In [ ]:
# out1 = np.array([0.25, 0.75])


In [ ]:
medians.shape,notmedian.shape

In [ ]:
df.head()

In [ ]:
df.iloc[notmedian,1:] = (0.2*df.iloc[notmedian,1:] + 0.7*np.repeat(np.expand_dims(quantsales.reshape(quantsales.shape[0]*quantsales.shape[1], order='F'),-1),28,1)[notmedian,:]
                       + 0.1*quantsalesW[notmedian,:])

df.iloc[medians,1:] = (0.8*df.iloc[medians,1:] + 0.2*np.repeat(np.expand_dims(quantsales.reshape(quantsales.shape[0]*quantsales.shape[1], order='F'),-1),28,1)[medians,:]
                       )

# # qs = np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995])
# out1 = np.array([0.25, 0.75])
# out2 = np.array([0.165, 0.835])
# out3 = np.array([0.025, 0.975])
# out4 = np.array([0.005, 0.995])

# out1_indexes = np.concatenate((np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[:274410,0]] )== out1[0])[0],np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[:274410,0]] )== out1[1])[0]))
# out2_indexes = np.concatenate((np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[:274410,0]] )== out2[0])[0],np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[:274410,0]] )== out2[1])[0]))
# out3_indexes = np.concatenate((np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[:274410,0]] )== out3[0])[0],np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[:274410,0]] )== out3[1])[0]))
# out4_indexes = np.concatenate((np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[:274410,0]] )== out4[0])[0],np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[:274410,0]] )== out4[1])[0]))
# df.iloc[out1_indexes,1:] = (0.175*df.iloc[out1_indexes,1:] + 0.725*np.repeat(np.expand_dims(quantsales.reshape(quantsales.shape[0]*quantsales.shape[1], order='F'),-1),28,1)[out1_indexes,:]
#                        + 0.1*quantsalesW[out1_indexes,:])
# df.iloc[out2_indexes,1:] = (0.2*df.iloc[out2_indexes,1:] + 0.7*np.repeat(np.expand_dims(quantsales.reshape(quantsales.shape[0]*quantsales.shape[1], order='F'),-1),28,1)[out2_indexes,:]
#                        + 0.1*quantsalesW[out2_indexes,:])
# df.iloc[out3_indexes,1:] = (0.225*df.iloc[out3_indexes,1:] + 0.675*np.repeat(np.expand_dims(quantsales.reshape(quantsales.shape[0]*quantsales.shape[1], order='F'),-1),28,1)[out3_indexes,:]
#                        + 0.1*quantsalesW[out3_indexes,:])
# df.iloc[out4_indexes,1:] = (0.25*df.iloc[out4_indexes,1:] + 0.65*np.repeat(np.expand_dims(quantsales.reshape(quantsales.shape[0]*quantsales.shape[1], order='F'),-1),28,1)[out4_indexes,:]
#                        + 0.1*quantsalesW[out4_indexes,:])


In [ ]:
df.head()

In [ ]:
# df

In [ ]:
# quantsalesdf=pd.DataFrame(quantsales)
# quantsalesdf['item_id']=sales['item_id'].values
# quantsalesdfGB=quantsalesdf.groupby('item_id',as_index=False).mean().iloc[:,1:]
# quantsalesdfGB

In [ ]:
# df.iloc[274410:274410+quantsalesdfGB.shape[0]*9,1:]

In [ ]:
# quantsalesdfGB.values.reshape(quantsalesdfGB.shape[0]*quantsalesdfGB.shape[1], order='F').shape

In [ ]:

# notmedian.shape

In [ ]:
# salesGitemQ=salesGitem.quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T
# salesGitemQ=salesGitemQ.values.reshape(salesGitemQ.shape[0]*salesGitemQ.shape[1], order='F')
# salesGitemQ=np.repeat(np.expand_dims(salesGitemQ,-1),28,1)

# # salesGitemQ=quantsalesdfGB.values.reshape(quantsalesdfGB.shape[0]*quantsalesdfGB.shape[1], order='F')
# # salesGitemQ=np.repeat(np.expand_dims(salesGitemQ,-1),28,1)

# # df.iloc[274410:274410+salesGitemQ.shape[0],1:] = 0.9*df.iloc[274410:274410+salesGitemQ.shape[0],1:]  +0.1*salesGitemQ
# medians=np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[274410:274410+salesGitemQ.shape[0],0]] )== 0.5)[0]
# notmedian=np.array([x for x in np.arange(274410,274410+salesGitemQ.shape[0]) if x not in medians+274410])
# df.iloc[notmedian,1:] = 0.9*df.iloc[notmedian,1:]  +0.1*salesGitemQ[notmedian-274410,:]
# # salesGitemQ.shape

In [ ]:
quantsales.shape

In [ ]:
df.iloc[:274410,1].shape,quantsales.shape

In [ ]:
# quantsales

In [ ]:
# df.iloc[:274410,1].values.reshape(30490,9, order='F')

In [ ]:
quantsalesdf=pd.DataFrame(quantsales)
# quantsalesdf=pd.DataFrame(df.iloc[:274410,1].values.reshape(30490,9, order='F'))
quantsalesdf['item_id']=sales['item_id'].values
quantsalesdf['state_id']=sales['state_id'].values
# quantsalesdfGB=(quantsalesdf.groupby(['state_id','item_id'],as_index=False).median().iloc[:,2:]+quantsalesdf.groupby(['state_id','item_id'],as_index=False).mean().iloc[:,2:])/2
quantsalesdfGB=quantsalesdf.groupby(['state_id','item_id'],as_index=False).mean().iloc[:,2:]
# quantsalesdfGB=quantsalesdfGB.sort_values(['state_id','item_id'])
quantsalesdfGB

In [ ]:
salesGitemQ=quantsalesdfGB.values.reshape(quantsalesdfGB.shape[0]*quantsalesdfGB.shape[1], order='F')
salesGitemQ=np.repeat(np.expand_dims(salesGitemQ,-1),28,1)
salesGitemQ.shape

In [ ]:

# df.iloc[274410:274410+salesGitemQ.shape[0],1:] = 0.9*df.iloc[274410:274410+salesGitemQ.shape[0],1:]  +0.1*salesGitemQ
medians=np.where(np.array([float(x.split('_')[-2]) for x in df.iloc[302067:302067+3049*3*9,0]] )== 0.5)[0]
notmedian=np.array([x for x in np.arange(302067,302067+3049*3*9) if x not in medians+302067])
df.iloc[notmedian,1:] = 0.91*df.iloc[notmedian,1:]  +0.09*salesGitemQ[notmedian-302067,:]

# df.iloc[medians,1:] = 0.98*df.iloc[medians,1:]  +0.02*salesGitemQ[medians-302067,:]

In [ ]:
# salesGitemQ[notmedian-302067,:].shape

In [ ]:
df.iloc[302067:302067+3049*3*9,:]#.shape

In [ ]:
# df.iloc[274410:274410+salesGitemQ.shape[0],:]

In [ ]:
# salesStateitem=sales.groupby(['item_id','state_id'])[['item_id','state_id']+[f"d_{i}" for i in range(914, 1914)]].sum()

In [ ]:
# salesStateitem

In [ ]:
# plt.hist(np.log1p(salesStateitem.values.flatten()),50)
# plt.show()

In [ ]:
# salesStateitemQ=np.log1p(salesStateitem.quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T)
# # salesStateitemQ=(salesStateitem.quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T)
# # salesStateitemQ=salesStateitemQ.values.reshape(salesStateitemQ.shape[0]*salesStateitemQ.shape[1], order='F')
# salesStateitemQ

In [ ]:
# # salesStateitemQ=salesStateitem.quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T
# salesStateitemQ=np.log1p(salesStateitem.quantile(np.array([0.005,0.025,0.165,0.25, 0.5, 0.75, 0.835, 0.975, 0.995]), axis=1).T)
# salesStateitemQ=salesStateitemQ.values.reshape(salesStateitemQ.shape[0]*salesStateitemQ.shape[1], order='F')
# salesStateitemQ=np.repeat(np.expand_dims(salesStateitemQ,-1),28,1)

# df.iloc[302067:302067+salesStateitemQ.shape[0],1:] = 0.6*df.iloc[302067:302067+salesStateitemQ.shape[0],1:]+0.4* salesStateitemQ

In [ ]:
# salesStateitemQ.shape

In [ ]:
# df.iloc[302067:302067+salesStateitemQ.shape[0],:] 

In [ ]:
# salesStateitemQ

In [ ]:
# salesStateitemQ.shape

In [ ]:
df.head()

In [ ]:
qals=np.array([float(x.split('_')[-2]) for x in df.id])

In [ ]:
qals

In [ ]:
df.shape, df.iloc[qals<0.5,1:8].shape, df.iloc[qals>0.5,1:8].shape, df.iloc[qals<0.5,8:15].shape

In [ ]:
# df.iloc[qals<0.5,1:15] =  df.iloc[qals<0.5,1:15]*1.01
# df.iloc[qals>0.5,1:15] =  df.iloc[qals>0.5,1:15]*0.99

# df.iloc[qals<0.5,15:] =  df.iloc[qals<0.5,15:]**1.0012 #- df.iloc[qals<0.5,15:]/1000##<------------------
# df.iloc[qals>0.5,15:] =  df.iloc[qals>0.5,15:]**1.001 #- df.iloc[qals>0.5,15:]/1000##<------------------

In [ ]:
# df.iloc[qals<0.5,15:] =  0.9*df.iloc[qals<0.5,15:].values+0.1*df.iloc[qals<0.5,1:15].values
# df.iloc[qals>0.5,15:] =  0.9*df.iloc[qals>0.5,15:].values+0.1*df.iloc[qals>0.5,1:15].values

In [ ]:
df.iloc[qals<0.5,1:15].shape, df.iloc[qals>0.5,15:].shape

In [ ]:
df.tail()

In [ ]:
df.head()

In [ ]:
df.iloc[:,1:].mean().mean()

In [ ]:
# np.clip(df.iloc[:,1:].values,0.0001,None).mean().mean()

In [ ]:
# df.iloc[:,1:]=np.clip(df.iloc[:,1:],0.0001,None)

In [ ]:
# df.iloc[:,1:].mean().mean()

In [ ]:
# len(df.iloc[:,1:].values[df.iloc[:,1:].values<=0.0001]),len(df.iloc[:,1:].values)

In [ ]:
plt.plot(df.iloc[:,1:].mean())
plt.show()
df.iloc[:,1:].mean()

In [ ]:
np.min(df)

In [ ]:
np.max(df)

In [ ]:
int(df.shape[0]/2)

In [ ]:
df.iloc[int(df.shape[0]/2):,1:]=df.iloc[:int(df.shape[0]/2),1:].values

In [ ]:
df.to_csv("submission01.csv", index = False)

In [ ]:
df.shape

In [ ]:
# df.iloc[302066:302066+82323,:]

In [ ]:
# df.loc[df.index >= len(df.index)//2, "id"] = df.loc[df.index >= len(df.index)//2, "id"].str.replace(
#                                     "_validation$", "_evaluation")

In [ ]:
df.shape

In [ ]:
42000*9*2

In [ ]:
df.iloc[:int(771120/2),1:].values-df.iloc[int(771120/2):,1:].values

In [ ]:
np.sum(df.iloc[:int(771120/2),1:].values-df.iloc[int(771120/2):,1:].values)

In [ ]:
df